In [1]:
import os
from os import listdir
import glob,progressbar,cv2
import tensorflow as tf
import numpy as np
from PIL import Image

2023-07-21 09:53:08.240406: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 09:53:08.704117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def normalize(input_images):
    hsv_image=tf.image.rgb_to_hsv(input_images)
    h,s,v=tf.split(hsv_image,num_or_size_splits=3,axis=-1)
    normalized_s=tf.image.per_image_standardization(s)
    # Concatenate the channels back together
    normalized_hsv_image = tf.concat([h, normalized_s, v], axis=-1)

    # Convert the image back to RGB color space
    normalized_images = tf.image.hsv_to_rgb(normalized_hsv_image)
    return normalized_images


In [3]:

def load_input_images(path_to_images: str) -> list:
    print(f"path_to_img inp: {path_to_images}")
    fire_class = os.path.basename(path_to_images)
    print(f"fire_class: {fire_class}")

    # Get all the images in the directory
    img_paths = []
    types = ["/*.jpg", "/*.png", "/*.jpeg"]
    for type in types:
        img_paths.extend(glob.glob(path_to_images + type))
    original_list=[]
    with progressbar.ProgressBar(max_value=len(img_paths)) as bar:
        for count, img_path in enumerate(img_paths):
            try:
                # read img
                imgRead = cv2.imread(img_path)
                # resize img
                imgRead = cv2.resize(imgRead, (224, 224))
                #imgRead = cv2.cvtColor(imgRead,cv2.COLOR_BGR2RGB)
                original_list.append(imgRead)
                
                bar.update(count)
            except Exception or Warning as err:
                print(f"Error in loading images: {err}")
                print(f"img Path that failed: {img_path}")
                bar.update(count)
                continue
    return original_list

# %%
pwd = os.getcwd()
print(pwd)
no_fire_directory_path = os.path.join(pwd, "clean_images/0")
fire_directory_path = os.path.join(pwd, "clean_images/1")

fire_list=load_input_images(fire_directory_path)
normalized_fire_images=normalize(np.array(fire_list))
no_fire_list=load_input_images(no_fire_directory_path)
normalized_no_fire_images=normalize(np.array(no_fire_list))

  0% (0 of 2273) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

/home/neureol/FD
path_to_img inp: /home/neureol/FD/clean_images/1
fire_class: 1


 18% (430 of 2273) |###                  | Elapsed Time: 0:00:02 ETA:   0:00:08libpng warning: iCCP: known incorrect sRGB profile
 29% (662 of 2273) |######               | Elapsed Time: 0:00:04 ETA:   0:00:10libpng warning: iCCP: known incorrect sRGB profile
 47% (1087 of 2273) |#########           | Elapsed Time: 0:00:07 ETA:   0:00:07Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
 97% (2216 of 2273) |################### | Elapsed Time: 0:00:21 ETA:   0:00:01Corrupt JPEG data: 1 extraneous bytes before marker 0xd9
100% (2273 of 2273) |####################| Elapsed Time: 0:00:22 Time:  0:00:22
2023-07-21 09:53:32.013199: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-21 09:53:32.028785: I tensorflow/compile

ResourceExhaustedError: {{function_node __wrapped__ConcatV2_N_3_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[2273,224,224,3] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ConcatV2] name: concat

In [ ]:

for count,img in enumerate(normalized_fire_images):
    img_path_to_save = os.path.join(
                    os.getcwd(),
                    "preprocessed_clean_images",
                    "normalized_fire_imgs",
                    "1",
                    f"image{count}.jpg",
                )
                
    cv2.imwrite(img_path_to_save, np.array(img))
    

In [ ]:
for count,img in enumerate(normalized_no_fire_images):
    img_path_to_save = os.path.join(
                    os.getcwd(),
                    "preprocessed_clean_images",
                    "normalized_fire_imgs",
                    "0",
                    f"image{count}.jpg",
                )
                
    cv2.imwrite(img_path_to_save, np.array(img))
    